<a href="https://colab.research.google.com/github/miklaszewskigrzegorz/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#matrix III ## 5 day

In [0]:
#!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%reload_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure


from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_tree/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_tree/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test  = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test , y_test  = test ['features'], test ['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

 

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callbacks = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callbacks]
      )
  
  return model

In [0]:

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:

def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [11]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(0.3),  
    
        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),  # 2 warstwy konwolucyjne obok siebie
        MaxPool2D(),
        Dropout(0.3),  

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),  # 2 warstwy konwolucyjne obok siebie
        MaxPool2D(),
        Dropout(0.3),  

        Flatten(),

        Dense(1024, activation='relu'),  
        Dropout(0.3), 

        Dense(1024, activation='relu'),  
        Dropout(0.3),   

        Dense(num_classes, activation='softmax'),  
                
      ])

#train_and_predict( get_cnn_v5(input_shape, num_classes) )
#get_cnn_v4(input_shape, num_classes).summary()

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.5151 - accuracy: 0.2887 - val_loss: 1.2025 - val_accuracy: 0.6305
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 0.9469 - accuracy: 0.6895 - val_loss: 0.2726 - val_accuracy: 0.9126
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.4057 - accuracy: 0.8726 - val_loss: 0.1293 - val_accuracy: 0.9568
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2557 - accuracy: 0.9223 - val_loss: 0.0577 - val_accuracy: 0.9837
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.1915 - accuracy: 0.9441 - val_loss: 0.0448 - val_accuracy: 0.9866


0.9594104308390022

In [25]:
model  = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train, params_fit={})
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.7698 - accuracy: 0.2545 - val_loss: 1.2799 - val_accuracy: 0.6289
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 1.1043 - accuracy: 0.6521 - val_loss: 0.4593 - val_accuracy: 0.8634
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.5628 - accuracy: 0.8192 - val_loss: 0.1761 - val_accuracy: 0.9435
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3598 - accuracy: 0.8876 - val_loss: 0.1058 - val_accuracy: 0.9665
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2585 - accuracy: 0.9201 - val_loss: 0.0641 - val_accuracy: 0.9799


0.9605442176870749

In [26]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1451 - accuracy: 0.9605


[0.14512920379638672, 0.960544228553772]

In [0]:
def func_obj(params):
  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

 
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=7,
      verbose=0,
      )
  
  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return { 'loss': - accuracy, 'status': STATUS_OK, 'model': model  }


In [28]:
space = {
    'batch-size': hp.quniform('batch_size', 50, 200, 20)

}

best = fmin (
    func_obj, 
    space, 
    tpe.suggest,
    1,
    Trials()



)

  1/138 [..............................]
 - ETA: 0s - loss: 5.7165 - accuracy: 0.0625

 10/138 [=>............................]
 - ETA: 0s - loss: 0.6017 - accuracy: 0.9000

 17/138 [==>...........................]
 - ETA: 0s - loss: 0.3729 - accuracy: 0.9375

 24/138 [====>.........................]
 - ETA: 0s - loss: 0.4909 - accuracy: 0.9115

 33/138 [======>.......................]
 - ETA: 0s - loss: 0.3632 - accuracy: 0.9356

 39/138 [=======>......................]
 - ETA: 0s - loss: 0.3115 - accuracy: 0.9439


In [0]:
# modyfikacja v5


In [31]:
def get_model(params):
  return Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),  
    
        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),  # 2 warstwy konwolucyjne obok siebie
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),  

        Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3,3), activation='relu'),  # 2 warstwy konwolucyjne obok siebie
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_tree']),  

        Flatten(),

        Dense(1024, activation='relu'),  
       Dropout(params['dropout_dense_block_one']), 

        Dense(1024, activation='relu'),  
        Dropout(params['dropout_dense_block_two']),    

        Dense(num_classes, activation='softmax'),  
                
      ])

ERROR! Session/line number was not unique in database. History logging moved to new session 68


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

 
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
      )
  
  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return { 'loss': - accuracy, 'status': STATUS_OK, 'model': model  }

In [36]:
space = {
    'batch-size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one':  hp.uniform('dropout_cnn_block_one',0,1),
    'dropout_cnn_block_two':  hp.uniform('dropout_cnn_block_two',0,1),
    'dropout_cnn_block_tree': hp.uniform('dropout_cnn_block_tree',0,1),
    'dropout_dense_block_one':hp.uniform('dropout_dense_block_one',0,1),
    'dropout_dense_block_two':hp.uniform('dropout_dense_block_two',0,1),

}

best = fmin (
    func_obj, 
    space, 
    tpe.suggest,
    30,
    Trials()
)

ERROR! Session/line number was not unique in
database. History logging moved to new session
72
  1/138 [..............................]
 - ETA: 0s - loss: 2.4697 - accuracy: 0.0000e+00

 10/138 [=>............................]
 - ETA: 0s - loss: 2.5153 - accuracy: 0.0000e+00

 19/138 [===>..........................]
 - ETA: 0s - loss: 2.4783 - accuracy: 0.1168    

 27/138 [====>.........................]
 - ETA: 0s - loss: 2.6095 - accuracy: 0.0822

 35/138 [======>.......................]
 - ETA: 0s - loss: 2.5247 - accuracy: 0.0634

 44/138 [========>............

In [37]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1451 - accuracy: 0.9605


0.960544228553772